In [ ]:
import os
import torch
from PIL import Image
from tqdm.auto import tqdm
from metrics import *
from dev import *


clean_path = "/fs/nexus-projects/HuangWM/datasets/main/diffusiondb/real"
attacked_path = "/fs/nexus-projects/HuangWM/datasets/attacked/diffusiondb/distortion_single_rotation-9-real"


QUALITY_METRICS = [
    "Legacy FID",
    "Clean FID",
    "CLIP FID",
    "PSNR",
    "SSIM",
    "NMI",
    "LPIPS",
    "WATSON",
    "Delta Aesthetics",
    "Delta Artifacts",
    "Delta CLIP-Score",
]

In [ ]:
compute_fid(
    clean_path,
    attacked_path,
    mode="legacy",
    device=torch.device("cuda"),
    num_workers=8,
    verbose=True,
)

In [ ]:
compute_fid(
    clean_path,
    attacked_path,
    mode="clean",
    device=torch.device("cuda"),
    num_workers=8,
    verbose=True,
)

In [ ]:
compute_fid(
    clean_path,
    attacked_path,
    mode="clip",
    device=torch.device("cuda"),
    num_workers=8,
    verbose=True,
)

In [ ]:
clean_images = [
    Image.open(os.path.join(clean_path, f"{idx}.png")) for idx in range(LIMIT)
]
attacked_images = [
    Image.open(os.path.join(attacked_path, f"{idx}.png")) for idx in range(LIMIT)
]

compute_psnr_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_ssim_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_nmi_repeated(clean_images, attacked_images, num_workers=8, verbose=True)

In [ ]:
compute_lpips_repeated(clean_images, attacked_images, mode="vgg", verbose=True)

In [ ]:
compute_watson_repeated(clean_images, attacked_images, mode="vgg", verbose=True)

In [ ]:
models = load_aesthetics_and_artifacts_models()

clean_aesthetics_scores, clean_artifaces_scores = [], []
for image in clean_images:
    aesthetics_score, artifaces_score = compute_aesthetics_and_artifacts_scores(
        image, models
    )
    clean_aesthetics_scores.append(aesthetics_score)
    clean_artifaces_scores.append(artifaces_score)


clean_scores = [
    compute_aesthetics_and_artifacts_scores(image, models) for image in clean_images
]

In [ ]:
models = load_open_clip_model_preprocess_and_tokenizer()
compute_clip_score(image, prompt, models)